In [88]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import fxcmpy
import os, sys 
import configparser
import datetime
import pandas as pd
import numpy as np

from structs import Pips, CurrencyPair
from strategy.fractals import FractalStrategy

In [141]:
all_times = []

start = datetime.datetime.strptime("00:00", "%H:%M")
end = datetime.datetime.strptime("20:55", "%H:%M")

t = start

while t <= end:
    all_times.append(t.strftime("%H:%M"))
    t += datetime.timedelta(minutes=5)

In [142]:
all_times

['00:00',
 '00:05',
 '00:10',
 '00:15',
 '00:20',
 '00:25',
 '00:30',
 '00:35',
 '00:40',
 '00:45',
 '00:50',
 '00:55',
 '01:00',
 '01:05',
 '01:10',
 '01:15',
 '01:20',
 '01:25',
 '01:30',
 '01:35',
 '01:40',
 '01:45',
 '01:50',
 '01:55',
 '02:00',
 '02:05',
 '02:10',
 '02:15',
 '02:20',
 '02:25',
 '02:30',
 '02:35',
 '02:40',
 '02:45',
 '02:50',
 '02:55',
 '03:00',
 '03:05',
 '03:10',
 '03:15',
 '03:20',
 '03:25',
 '03:30',
 '03:35',
 '03:40',
 '03:45',
 '03:50',
 '03:55',
 '04:00',
 '04:05',
 '04:10',
 '04:15',
 '04:20',
 '04:25',
 '04:30',
 '04:35',
 '04:40',
 '04:45',
 '04:50',
 '04:55',
 '05:00',
 '05:05',
 '05:10',
 '05:15',
 '05:20',
 '05:25',
 '05:30',
 '05:35',
 '05:40',
 '05:45',
 '05:50',
 '05:55',
 '06:00',
 '06:05',
 '06:10',
 '06:15',
 '06:20',
 '06:25',
 '06:30',
 '06:35',
 '06:40',
 '06:45',
 '06:50',
 '06:55',
 '07:00',
 '07:05',
 '07:10',
 '07:15',
 '07:20',
 '07:25',
 '07:30',
 '07:35',
 '07:40',
 '07:45',
 '07:50',
 '07:55',
 '08:00',
 '08:05',
 '08:10',
 '08:15',


In [94]:
# to pull off the unfinished date
now = datetime.datetime.utcnow()
closest = now - datetime.timedelta(minutes=now.minute % 5,
                                   seconds=now.second,
                                   microseconds=now.microsecond)

In [95]:
closest

datetime.datetime(2020, 11, 30, 18, 30)

In [96]:
print(now)
print(closest)

2020-11-30 18:33:27.570835
2020-11-30 18:30:00


In [3]:
abspath_api = os.path.abspath('configs/api.ini')

config = configparser.ConfigParser()
config.read(abspath_api)

fxcm_section = config['FXCM']

access_token = fxcm_section['access_token']
log_file = fxcm_section['log_file']
log_level = fxcm_section['log_level']

con = fxcmpy.fxcmpy(access_token=access_token, log_level=log_level)
account_id = con.get_default_account()
print(f"FXCM API connection established")

FXCM API connection established


In [4]:
RENAMER = {'bidopen': 'BidOpen', 'bidhigh': 'BidHigh', 'bidlow': 'BidLow', 'bidclose': 'BidClose',
           'askopen': 'AskOpen', 'askhigh': 'AskHigh', 'asklow': 'AskLow', 'askclose': 'AskClose'}

pair = CurrencyPair('EURUSD')
jpy_pair = pair.jpy_pair

con.subscribe_market_data(pair.fxcm_name)

In [5]:
target_level = 4.0
back_level = 2.1
break_level = Pips(2, jpy_pair)
sl_extension = Pips(1, jpy_pair)
max_width = Pips(12, jpy_pair)
min_width = Pips(2, jpy_pair)
risk = 0.015

strategy = FractalStrategy(target_level=target_level,
                           back_level=back_level,
                           break_level=break_level,
                           sl_extension=sl_extension,
                           max_width=max_width,
                           min_width=min_width,
                           risk=risk)

In [119]:
data = con.get_candles(instrument=pair.fxcm_name, period='m5', number=24)
data = data.rename(RENAMER, axis=1)

last_price = con.get_last_price(pair.fxcm_name)
latest_price = (last_price['Bid'] + last_price['Ask'])/2

upper_fractal, lower_fractal = strategy.get_fractals(data)

if upper_fractal is not None and lower_fractal is not None:
    target_l, back_l, entry_l, sl_l = strategy.get_long_order(upper_fractal, lower_fractal)
    target_s, back_s, entry_s, sl_s = strategy.get_short_order(upper_fractal, lower_fractal)
    
    if upper_fractal > latest_price > lower_fractal:
        print('Price is between fractals')
    
(upper_fractal, lower_fractal)

(None, None)

In [126]:
last_price

Bid     1.19558
Ask     1.19565
High    1.20035
Low     1.19417
Name: 2020-11-30 18:38:02.097000, dtype: float64

In [168]:
now = datetime.datetime.utcnow()
closest = now - datetime.timedelta(minutes=(now.minute%5), seconds=now.second, microseconds=now.microsecond)
closest += datetime.timedelta(minutes=5)

In [169]:
print(now)
print(closest)

2020-11-30 19:40:46.527210
2020-11-30 19:45:00


In [193]:
from typing import List

class ScheduleHelper:

    day_map = {'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}

    def __init__(self, time_now, frequency):

        self._time_now = time_now
        self._frequency = frequency

    def get_schedule(self, day: str):
        print(self.weeknumber_now)
        if self.weeknumber_now < ScheduleHelper.day_map[day]:
            return self.get_time_intervals('00:00', '23:55')
        elif self.weeknumber_now == ScheduleHelper.day_map[day]:
            print('here')
            next_time = self._time_now - datetime.timedelta(minutes=(self._time_now.minute % 5),
                                                            seconds=self._time_now.second,
                                                            microseconds=self._time_now.microsecond)
            next_time += datetime.timedelta(minutes=5)
            return self.get_time_intervals(next_time.strftime('%H:%M'), '23:55')
        else:
            return []

    @property
    def monday(self) -> List[str]:
        return self.get_schedule('Monday')

    @property
    def tuesday(self) -> List[str]:
        return self.get_schedule('Tuesday')

    @property
    def wednesday(self) -> List[str]:
        return self.get_schedule('Wednesday')

    @property
    def thursday(self) -> List[str]:
        return self.get_schedule('Thursday')

    @property
    def friday(self) -> List[str]:
        return self.get_schedule('Friday')

    @property
    def weeknumber_now(self) -> int:
        return int(self._time_now.strftime('%w')) + 1

    @staticmethod
    def get_time_intervals(start: str, end: str) -> List[str]:
        all_times = []
        print(start)
        start = datetime.datetime.strptime(start, "%H:%M")
        end = datetime.datetime.strptime(end, "%H:%M")

        t = start

        while t <= end:
            all_times.append(t.strftime("%H:%M"))
            t += datetime.timedelta(minutes=5)

        return all_times

In [194]:
helper = ScheduleHelper(datetime.datetime.utcnow(), 'm5')

In [195]:
helper.monday

2


[]

In [45]:
historical_price = data

processed = pd.DataFrame()

processed['High'] = (historical_price['BidHigh'] + historical_price['AskHigh']) / 2
processed['Low'] = (historical_price['BidLow'] + historical_price['AskLow']) / 2

processed['UpperFractal'] = np.where(
    (processed['High'] > processed['High'].shift(1)) & (processed['High'] > processed['High'].shift(-1)), True,
    False)

processed['LowerFractal'] = np.where(
    (processed['Low'] < processed['Low'].shift(1)) & (processed['Low'] < processed['Low'].shift(-1)), True,
    False)

all_upper = processed[processed['UpperFractal'] == True]['High']
all_lower = processed[processed['LowerFractal'] == True]['Low']

if len(all_upper) == 0 or len(all_lower) == 0:
    None, None

upper_fractal = all_upper.iloc[-1]
lower_fractal = all_lower.iloc[-1]

In [48]:
upper_fractal - lower_fractal

0.0017449999999998855

In [47]:
lower_fractal

1.198565

In [32]:
con.get_accounts_summary()['usableMargin3'][0]

42572.58

In [78]:
oco_order = con.create_oco_order(symbol=pair.fxcm_name, is_buy=True, is_buy2=False,
                                 amount=1, is_in_pips=False, time_in_force='GTC',
                                 rate=1.3, rate2=1.1, order_type='MarketRange', at_market=0)

In [81]:
for order in con.get_orders(kind='list'):
    order.delete()

AttributeError: 'dict' object has no attribute 'delete'

In [82]:
for order in oco_order.get_orders():
    order.delete()

In [85]:
oco_order.get_orders()

[]

In [75]:
order??

Type:        fxcmpy_order
String form:
accountId:        829763
accountName:      00829763
amountK:          1
buy:              0
currency:         EUR/USD
currencyPoint:    0.07478
expireDate:       
isBuy:            False
isELSOrder:       False
isEntryOrder:     True
isLimitOrder:     False
isNetQuantity:    False
isStopOrder:      True
limit:            0
limitPegBaseType: -1
limitRate:        0
ocoBulkId:        210847835
orderId:          210847836
range:            0
sell:             1.1
status:           Waiting
stop:             0
stopMove:         0
stopPegBaseType:  -1
stopRate:         0
time:             2020-11-30 15:23:26.064000
timeInForce:      GTC
tradeId:          119911336
type:             SE
File:        c:\users\dema\anaconda3\envs\rl_fx\lib\site-packages\fxcmpy\fxcmpy_order.py
Source:     
class fxcmpy_order(object):
    """ A class to realize entry orders of the FXCM API.

    Caution:

    Do not initialize fxcm order object manually, these orders will not


In [53]:
entry_order = con.create_entry_order(symbol=pair.fxcm_name, is_buy=True, order_type='Entry',
                                     rate=1., stop=, limit=1, amount=1, time_in_force='GTC')

NameError: name 'stop' is not defined

In [ ]:
entry_order.delete()

In [ ]:
con.is_connected()

In [ ]:
con.get_orders().T

In [60]:
con.get_orders(kind='list')

[{'t': 3,
  'ratePrecision': 5,
  'orderId': '210847834',
  'tradeId': '119911335',
  'time': '11302020152326063',
  'accountName': '00829763',
  'accountId': '829763',
  'timeInForce': 'GTC',
  'expireDate': '',
  'currency': 'EUR/USD',
  'isBuy': True,
  'buy': 1.3,
  'sell': 0,
  'type': 'SE',
  'status': 1,
  'amountK': 1,
  'currencyPoint': 0.07478,
  'stopMove': 0,
  'stop': 0,
  'stopRate': 0,
  'limit': 0,
  'limitRate': 0,
  'isEntryOrder': True,
  'ocoBulkId': 210847835,
  'isNetQuantity': False,
  'isLimitOrder': False,
  'isStopOrder': True,
  'isELSOrder': False,
  'stopPegBaseType': -1,
  'limitPegBaseType': -1,
  'range': 0},
 {'t': 3,
  'ratePrecision': 5,
  'orderId': '210847836',
  'tradeId': '119911336',
  'time': '11302020152326064',
  'accountName': '00829763',
  'accountId': '829763',
  'timeInForce': 'GTC',
  'expireDate': '',
  'currency': 'EUR/USD',
  'isBuy': False,
  'buy': 0,
  'sell': 1.1,
  'type': 'SE',
  'status': 1,
  'amountK': 1,
  'currencyPoint': 0.